In [50]:
# Hyperparams

import torch.nn as nn
import torch

# training config
NUM_EPOCHS= 1
LR=0.001
# dataset config
batch_size = 64
generator=torch.Generator().manual_seed(42) # Can be included for reproducability

In [51]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [52]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [66]:
_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

NUM_CLASSES = 0

def getTrainingSet(dataset_name):
  if dataset_name == 'CIFAR-10':
    print("in cifar-10")
    NUM_CLASSES=10

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform_train)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=transform_test)
    

    trainset, validset = torch.utils.data.random_split(trainset, 
                                                      [int(len(trainset)*0.8),len(trainset)- 
                                                      int(len(trainset)*0.8)], generator=generator)
    
  elif dataset_name == 'tiny-imagenet':
    NUM_CLASSES=200
    
    !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
    !unzip -qq 'tiny-imagenet-200.zip'
    
    
    totalset = torchvision.datasets.ImageFolder('tiny-imagenet-200/train', 
                                                   transform=transform_train)
  
    train_counts = [0] * 200
    valid_counts = [0] * 200
    trainset = []
    validset = []
    testset = []
    for item in totalset:
        if train_counts[item[1]] < 350:
            trainset.append(item)
            train_counts[item[1]] += 1
        elif valid_counts[item[1]] < 75:
            validset.append(item)
            valid_counts[item[1]] += 1
        else:
            testset.append(item)

  elif dataset_name == 'Caltech101':
    NUM_CLASSES=101
    !gdown https://drive.google.com/uc?id=1DX_XeKHn3yXtZ18DD7qc1wf-Jy5lnhD5
    !unzip -qq '101_ObjectCategories.zip' 

    PATH = '101_ObjectCategories/'

    transform = transforms.Compose(
      [transforms.CenterCrop(256),
      transforms.Resize((64,64)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    totalset = torchvision.datasets.ImageFolder(PATH, transform=transform_train)

    X, y = zip(*totalset)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, 
                                                      stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, 
                                                    test_size = 0.5, 
                                                    stratify=y_val)

    trainset, validset, testset = list(zip(X_train, y_train)), list(zip(X_val, y_val)), list(zip(X_test, y_test))




  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                            shuffle=False,num_workers=2)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  return trainset, testset, trainloader, testloader

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
class ResNetBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(ResNetBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [56]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [57]:
def ResNet18():
    NUM_CLASSES=200
    return ResNet(ResNetBasicBlock, [2,2,2,2], NUM_CLASSES)

In [58]:
# Model
net = ResNet18()
trainset, testset, trainloader, testloader = getTrainingSet("tiny-imagenet")


In [59]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [60]:
def train(epoch):
  net.train()
  correct_images = 0
  total_images = 0
  training_loss = 0
  for batch_index, (images, labels) in enumerate(trainloader):
    optimizer.zero_grad()
    outputs = net(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
    _, predicted = outputs.max(1)
    total_images += labels.size(0)
    correct_images += predicted.eq(labels).sum().item()
    print('Epoch: %d, Batch: %d, Loss: %.3f, '
                  'Accuracy: %.3f%% (%d/%d)' % (epoch, batch_index, training_loss/(batch_index+1),
                                           100.*correct_images/total_images, correct_images, total_images))

In [61]:
def test():
    test_loss = 0
    total_images = 0
    correct_images = 0
    net.eval()
    with torch.no_grad():
      for batch_index, (images, labels) in enumerate(testloader):
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total_images += labels.size(0)
        correct_images += predicted.eq(labels).sum().item()
        print(batch_index, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                  % (test_loss/(batch_index+1), 100.*correct_images/total_images, correct_images, total_images))
        test_accuracy = 100.*correct_images/total_images
        print("accuracy of test set is",test_accuracy)

In [65]:
for epoch in range(NUM_EPOCHS):
  train(epoch) 
  #In 1 epoch 48% accuracy

Epoch: 0, Batch: 0, Loss: 5.365, Accuracy: 0.000% (0/64)
Epoch: 0, Batch: 1, Loss: 5.404, Accuracy: 0.000% (0/128)
Epoch: 0, Batch: 2, Loss: 5.408, Accuracy: 0.521% (1/192)
Epoch: 0, Batch: 3, Loss: 5.412, Accuracy: 0.391% (1/256)
Epoch: 0, Batch: 4, Loss: 5.405, Accuracy: 0.312% (1/320)
Epoch: 0, Batch: 5, Loss: 5.387, Accuracy: 0.260% (1/384)
Epoch: 0, Batch: 6, Loss: 5.387, Accuracy: 0.223% (1/448)
Epoch: 0, Batch: 7, Loss: 5.386, Accuracy: 0.195% (1/512)
Epoch: 0, Batch: 8, Loss: 5.406, Accuracy: 0.174% (1/576)
Epoch: 0, Batch: 9, Loss: 5.414, Accuracy: 0.156% (1/640)
Epoch: 0, Batch: 10, Loss: 5.404, Accuracy: 0.142% (1/704)
Epoch: 0, Batch: 11, Loss: 5.398, Accuracy: 0.260% (2/768)
Epoch: 0, Batch: 12, Loss: 5.395, Accuracy: 0.240% (2/832)


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/Users/samanthahay/miniconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/Users/samanthahay/miniconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/samanthahay/miniconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/Users/samanthahay/miniconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [64]:
test()

0 235 Loss: 5.313 | Acc: 0.000% (0/64)
accuracy of test set is 0.0
1 235 Loss: 5.310 | Acc: 0.000% (0/128)
accuracy of test set is 0.0
2 235 Loss: 5.304 | Acc: 0.000% (0/192)
accuracy of test set is 0.0
3 235 Loss: 5.302 | Acc: 0.000% (0/256)
accuracy of test set is 0.0
4 235 Loss: 5.303 | Acc: 0.000% (0/320)
accuracy of test set is 0.0
5 235 Loss: 5.303 | Acc: 0.000% (0/384)
accuracy of test set is 0.0
6 235 Loss: 5.299 | Acc: 0.000% (0/448)
accuracy of test set is 0.0
7 235 Loss: 5.304 | Acc: 0.000% (0/512)
accuracy of test set is 0.0
8 235 Loss: 5.303 | Acc: 0.000% (0/576)
accuracy of test set is 0.0
9 235 Loss: 5.299 | Acc: 0.000% (0/640)
accuracy of test set is 0.0
10 235 Loss: 5.296 | Acc: 0.000% (0/704)
accuracy of test set is 0.0
11 235 Loss: 5.295 | Acc: 0.000% (0/768)
accuracy of test set is 0.0
12 235 Loss: 5.295 | Acc: 0.000% (0/832)
accuracy of test set is 0.0
13 235 Loss: 5.299 | Acc: 0.000% (0/896)
accuracy of test set is 0.0
14 235 Loss: 5.299 | Acc: 0.000% (0/960)
accu

KeyboardInterrupt: 